In [1]:
!pip install wandb
!pip install torch
!pip install lightning
!pip install matplotlib
!pip install tqdm
!pip install sentence-transformers

  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 2.8 MB/s eta 0:00:00a 0:00:01
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 2.7 MB/s eta 0:00:00


In [1]:
# !wandb login 6ff71c9207e0564c7e57727b36e7272a588ea599 # Luke
!wandb login 76a5fa31b18277c7da467875faee9a595d539767 # Henri

wandb: Appending key for api.wandb.ai to your netrc file: /Users/henrismidt/.netrc


# Training of Text-to-Image network translation

Folders to upload:

- autoencoder
  - *
- cINN
  - *
- datasets
  - *
- models
  - ae_100.pth
- utils
  - *


In [5]:
import torch
import lightning as L
from pytorch_lightning.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer
import wandb


from .datasets.encoded_mnist import EncodedMNIST
from cINN import ConditionalRealNVP, OurConditionalRealNVP
from autoencoder import AutoencoderSimple
from utils import get_best_device

ImportError: attempted relative import with no known parent package

## Load pretrained models

In [2]:
embedding_model = SentenceTransformer("intfloat/multilingual-e5-small")

In [3]:
ae_path = "./models/ae_100.pth"
autoencoder = AutoencoderSimple()
autoencoder.load_state_dict(torch.load(ae_path, map_location=get_best_device()))

<All keys matched successfully>

## Load data

In [6]:
train_data = EncodedMNIST(autoencoder=autoencoder, embedding_model=embedding_model, train=True)

TRAIN_SIZE = int(0.8 * len(train_data))
VAL_SIZE = len(train_data) - TRAIN_SIZE

train_data, val_data = torch.utils.data.random_split(train_data, [TRAIN_SIZE, VAL_SIZE])

train_loader = DataLoader(train_data, batch_size=100, shuffle=True)
val_loader = DataLoader(val_data, batch_size=100, shuffle=True)

Encoding:   0%|          | 0/120 [00:00<?, ?it/s]

500


Encoding:   1%|          | 1/120 [05:39<11:12:34, 339.12s/it]

500


Encoding:   2%|▏         | 2/120 [05:48<4:45:40, 145.26s/it] 

...

Encoding:  98%|█████████▊| 118/120 [39:40<00:23, 11.81s/it]

500


Encoding:  99%|█████████▉| 119/120 [39:51<00:11, 11.45s/it]

500


Encoding: 100%|██████████| 120/120 [40:04<00:00, 20.04s/it]


# Train cINN with WandB Sweep

In [9]:
# Define the sweep configuration
sweep_config = {
  "method": "grid", 
  "metric": { 
    'name': 'val_loss',
    "goal": "minimize"  
  },
  "parameters": {
    "hidden_size": {"values": [64, 128, 256]},  
    "n_blocks": {"values": [10, 20, 30]},  
    "epochs": {"values": [20, 35, 50]},
  }
}

# Function to be called for each sweep
def train():
    # Initialize a W&B run
    wandb.init()

    # Access sweep parameters
    config = wandb.config
    hidden_size = config.hidden_size
    n_blocks = config.n_blocks
    epochs = config.epochs

    # Model initialization (modified to use config parameters)
    cinn = ConditionalRealNVP(input_size=64, hidden_size=hidden_size, n_blocks=n_blocks, condition_size=384)

    # Initialize the WandbLogger
    wandb_logger = WandbLogger(project="cINN-Text-to-Image", entity="network-to-network")

    # Define the ModelCheckpoint callback
    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        dirpath='models/',
        filename='cINN-{epoch:02d}-{val_loss:.2f}',
        save_top_k=1,
        mode='min',
    )

    # Trainer initialization
    trainer = L.Trainer(
        max_epochs=epochs,
        logger=wandb_logger,
        callbacks=[checkpoint_callback]  # Include checkpoint callback here
    )

    # Fit the model
    trainer.fit(model=cinn, train_dataloaders=train_loader, val_dataloaders=val_loader)

    # Optionally, you can log the best model's file path to W&B
    wandb.log({'best_model_path': checkpoint_callback.best_model_path})

    # Close the W&B run when done
    wandb.finish()


In [10]:
# Initialize sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="cINN-Text-to-Image", entity="network-to-network")

# Start sweep
wandb.agent(sweep_id, train, count=27)  # Adjust count as needed


Create sweep with ID: 548n69u9
Sweep URL: https://wandb.ai/network-to-network/cINN-Text-to-Image/sweeps/548n69u9


wandb: Agent Starting Run: qs82o5fp with config:
wandb: 	epochs: 20
wandb: 	hidden_size: 64
wandb: 	n_blocks: 10
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Train cINN

In [9]:
hidden_size = 128
n_blocks = 20
epochs = 100

cinn = ConditionalRealNVP(input_size=64, hidden_size=hidden_size, n_blocks=n_blocks, condition_size=384)

hyperparams = {
    "hidden_size": hidden_size,
    "n_blocks": n_blocks,
    "epochs": epochs
}

wandb_logger = WandbLogger(project="cINN-Text-to-Image",
                           entity="network-to-network") #optionally pass in run name with: name="test-run"
trainer = L.Trainer(max_epochs=epochs, logger=wandb_logger)
trainer.fit(model=cinn, train_dataloaders=train_loader, val_dataloaders=val_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
wandb: Currently logged in as: finnhenri-smidt (network-to-network). Use `wandb login --relogin` to force relogin


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name                | Type          | Params
------------------------------------------------------
0 | coupling_blocks     | ModuleList    | 1.6 M 
1 | orthogonal_matrices | ParameterList | 77.8 K
------------------------------------------------------
1.6 M     Trainable params
77.8 K    Non-trainable params
1.6 M     Total params
6.563     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name                | Type          | Params
------------------------------------------------------
0 | coupling_blocks     | ModuleList    | 1.6 M 
1 | orthogonal_matrices | ParameterList | 77.8 K
------------------------------------------------------
1.6 M     Trainable params
77.8 K    Non-trainable params
1.6 M     Total params
6.563     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [10]:
torch.save(cinn.state_dict(), 'cinn_100.pth')
